# Predicción de despacho de productos
## Alternativa 1: Modelo por cada producto

In [10]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
from skopt.space import Integer, Categorical


In [4]:
pivot_df = pd.read_csv('../data/pivot_df_interpolado.csv')

In [5]:
pivot_df.shape

(262805, 38)

In [6]:
# Drop rows where all columns except 'product_id' and 'customer_id' are null
cols_to_check = pivot_df.columns.difference(['product_id', 'customer_id'])
pivot_df = pivot_df.dropna(subset=cols_to_check, how='all')
pivot_df.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,20001,10002,35.72806,6.79415,29.94128,22.81133,31.22847,47.57025,21.84874,17.08052,...,31.97079,55.41679,30.87299,144.07021,37.14616,0.00000,72.08551,17.40806,45.61495,113.33165
2,20001,10003,143.49426,20.48319,137.87537,68.89292,135.12190,171.01785,64.66196,83.63410,...,170.89924,230.00152,1.84835,0.00000,138.23391,162.07198,233.20532,76.00625,86.14415,102.27517
3,20001,10004,184.72927,104.03894,295.43924,247.65632,188.37819,195.02683,379.44270,237.16848,...,102.64484,91.67799,389.02653,66.71971,228.62366,96.11402,288.34205,324.96172,195.67828,34.64810
4,20001,10005,19.08407,5.17117,5.17117,0.86186,37.95546,19.08407,43.35049,67.53856,...,6.90049,22.18016,15.89578,0.00000,8.25595,0.00000,12.80400,17.13921,12.22149,19.60368


In [7]:
pivot_df.shape

(262805, 38)

In [12]:
from sklearn.preprocessing import StandardScaler

# Selecciona solo las columnas de las series temporales (excluye product_id y customer_id)
series_cols = pivot_df.columns.difference(['product_id', 'customer_id'])

# Inicializa el scaler
scaler = StandardScaler()

# Ajusta y transforma las columnas de series temporales
pivot_df[series_cols] = scaler.fit_transform(pivot_df[series_cols])

pivot_df.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,34.011090,91.505733,34.227381,5.620967,43.017729,53.427125,42.221730,20.868623,...,75.994505,197.630988,222.727019,41.281357,84.468477,25.934477,61.055903,79.241259,129.845076,116.150285
1,20001,10002,12.111477,2.983680,10.997802,9.722515,13.219633,19.782447,9.049391,8.162839,...,18.548231,29.995683,15.568913,90.310715,21.615563,-0.073820,40.334746,7.781788,24.972975,73.018654
2,20001,10003,49.154605,9.293417,51.098561,29.588490,57.586964,71.391232,26.945608,40.365988,...,99.508313,124.723610,0.869164,-0.080137,80.641964,125.230207,130.647599,34.180412,47.221556,65.889035
3,20001,10004,63.328565,47.807064,109.638275,106.654172,80.329879,81.428481,158.526102,114.656364,...,59.733293,49.670648,196.958487,41.780366,133.421669,74.235593,161.553515,146.335444,107.350521,22.280652
4,20001,10005,6.390337,2.235594,1.794961,0.260003,16.092371,7.873450,18.037269,32.577871,...,3.938598,11.961821,7.983592,-0.080137,4.746208,-0.073820,7.105547,7.660670,6.641573,12.579465


In [17]:
from itertools import product
from skopt import BayesSearchCV
from skopt.space import Categorical
from sklearn.base import BaseEstimator, RegressorMixin

i = 20001
# 20001.0	10005.0	
serie = pivot_df[(pivot_df['product_id'] == 20001) & (pivot_df['customer_id'] == 10001)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
# serie = pivot_df[(pivot_df['product_id'] == 20001) & (pivot_df['customer_id'] == 10005)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
serie = serie.dropna()

# Convert index to datetime for SARIMAX
# Assuming your index is in 'YYYYMM' format
serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
serie = serie.sort_index()

# Define training and testing periods
train_end_date = '2019-10-31'
test_date = '2019-12-01' # For prediction

# Split data
train_data = serie[serie.index <= train_end_date]
test_data = serie[serie.index == test_date]
# Wrapper para SARIMAX compatible con sklearn
class SARIMAXWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, order=(1,1,1), seasonal_order=(1,1,0,4)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model_ = None
        self.model_fit_ = None

    def fit(self, X, y):
        self.model_ = SARIMAX(y, order=self.order, seasonal_order=self.seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        self.model_fit_ = self.model_.fit(disp=False)
        return self

    def predict(self, X):
        dates = pd.to_datetime(np.array(X).flatten())
        preds = []
        for date in dates:
            pred = self.model_fit_.predict(start=date, end=date)
            preds.append(pred.iloc[0])
        return np.array(preds)

# Espacio de búsqueda
param_space = {
    'order': Categorical([
        (0,0,0), (0,0,1), (0,1,0), (0,1,1), (0,2,1),
        (1,0,0), (1,0,1), (1,1,0), (1,1,1), (1,1,2), (1,2,1), (1,2,2),
        (2,0,0), (2,0,1), (2,0,2), (2,1,0), (2,1,1), (2,1,2), (2,2,1), (2,2,2),
        (3,0,0), (3,0,1), (3,1,0), (3,1,1), (3,1,2), (3,2,1), (3,2,2)
    ]),
    'seasonal_order': Categorical([
        (0,0,0,4), (0,1,0,4), (1,0,0,4), (1,1,0,4), (1,1,1,4), (2,1,0,4), (0,1,1,4), (1,0,1,4), (2,0,1,4),
        (1,2,0,4), (2,2,0,4), (2,2,1,4), (1,2,1,4), (2,1,1,4), (2,0,2,4), (1,0,2,4), (0,2,1,4), (0,2,0,4),
        (0,0,0,12), (0,1,0,12), (1,0,0,12), (1,1,0,12), (1,1,1,12), (2,1,0,12), (0,1,1,12), (1,0,1,12), (2,0,1,12),
        (1,2,0,12), (2,2,0,12), (2,2,1,12), (1,2,1,12), (2,1,1,12), (2,0,2,12), (1,0,2,12), (0,2,1,12), (0,2,0,12)
    ])
}

# Prepara los datos para sklearn (X: fechas, y: valores)
X_train_sklearn = train_data.index.values.reshape(-1, 1)
y_train_sklearn = train_data.values

opt = BayesSearchCV(
    SARIMAXWrapper(),
    param_space,
    n_iter=20,
    cv=[(np.arange(len(X_train_sklearn)), np.arange(len(X_train_sklearn)))],  # No split, solo fit
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=1
)

opt.fit(X_train_sklearn, y_train_sklearn)

# Mejor modelo
best_model = opt.best_estimator_
best_params = (best_model.order, best_model.seasonal_order)

# Predicción para diciembre 2019
pred_201912 = best_model.predict(np.array([pd.to_datetime(test_date)]))[0]

print(f"Mejor combinación: order={best_params[0]}, seasonal_order={best_params[1]}")
print(f"Predicción para product_id={i} en 2019-12: {pred_201912:.5f}")
if not test_data.empty:
    print(f"Valor real: {test_data.iloc[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
i = 20001
# 20001.0	10005.0	
# serie = pivot_df_interpolado[(pivot_df_interpolado['product_id'] == 20001) & (pivot_df_interpolado['customer_id'] == 10001)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
serie = pivot_df_interpolado[(pivot_df_interpolado['product_id'] == 20001) & (pivot_df_interpolado['customer_id'] == 10005)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
serie = serie.dropna()

# Convert index to datetime for SARIMAX
# Assuming your index is in 'YYYYMM' format
serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
serie = serie.sort_index()

# Define training and testing periods
train_end_date = '2019-10-31'
test_date = '2019-12-01' # For prediction

# Split data
train_data = serie[serie.index <= train_end_date]
test_data = serie[serie.index == test_date]



param_space = {
    'order': Categorical([
        (1,1,1), (2,1,1), (1,1,2), (2,1,2), (0,1,1), (1,0,1)
    ]),
    'seasonal_order': Categorical([
        (1,1,0,4), (1,1,1,4), (2,1,0,4), (0,1,1,4), (1,0,1,4),
        (1,1,0,12), (1,1,1,12), (2,1,0,12)
    ])
}



# Initialize and fit the SARIMAX model
# 'enforce_stationarity=False' and 'enforce_invertibility=False' can help with convergence
model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit(disp=False) # disp=False to suppress optimization output

# Make prediction for December 2019
start_index = pd.to_datetime(test_date)
end_index = pd.to_datetime(test_date)

# Predict using the fitted model
pred_201912 = model_fit.predict(start=start_index, end=end_index)

print(f"Predicción para product_id={i} en 2019-12: {pred_201912.iloc[0]:.5f}")

if not test_data.empty:
    print(f"Valor real: {test_data.iloc[0]:.5f}")
    # Optional: Calculate and print RMSE for evaluation
    # rmse = np.sqrt(mean_squared_error(test_data, pred_201912))
    # print(f"RMSE: {rmse:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
df_first_2500 = pivot_df_interpolado.head(2500).copy()
df_first_2500.head()

In [ ]:
order = (1,1,1)
seasonal_order = (2,1,0,4) 

In [ ]:
resultados = []

for idx, row in df_first_2500.iterrows():
    print(f"Processing index: {idx}")
    product_id = row['product_id']
    customer_id = row['customer_id']
    serie = row[2:]
    serie = serie.dropna()
    if len(serie) < 8 or '201912' not in serie.index.astype(str):
        continue

    # Convertir el índice a fechas
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
    serie = serie.sort_index()

    # Definir periodos de train y test
    train_end_date = '2019-10-31'
    test_date = '2019-12-01'

    train_data = serie[serie.index <= train_end_date]
    test_data = serie[serie.index == test_date]

    if len(train_data) < 8 or test_data.empty:
        continue

    try:
        model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        pred_201912 = model_fit.predict(start=pd.to_datetime(test_date), end=pd.to_datetime(test_date)).iloc[0]
        real_201912 = test_data.iloc[0]
        resultados.append({
            'product_id': product_id,
            'customer_id': customer_id,
            'sarimax_pred_201912': pred_201912,
            'real_201912': real_201912
        })
    except Exception:

        continue

df_resultados = pd.DataFrame(resultados)
df_resultados.head()


In [ ]:
df_resultados.head(200)

In [ ]:
# Calcula el error porcentual absoluto medio (MAPE) en sarimax_pred_201912
mape_df_resultados = np.mean(np.abs((df_resultados['real_201912'] - df_resultados['sarimax_pred_201912']) / df_resultados['real_201912'])) * 100
print(f"Error porcentual absoluto medio (MAPE) en df_resultados: {mape_df_resultados:.2f}%")

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from sklearn.base import BaseEstimator, RegressorMixin

# Wrapper para SARIMAX compatible con sklearn
class SARIMAXWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, order=(1,1,1), seasonal_order=(1,1,0,4)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model_ = None
        self.model_fit_ = None

    def fit(self, X, y):
        # SARIMAX espera un índice de fechas
        self.model_ = SARIMAX(y, order=self.order, seasonal_order=self.seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        self.model_fit_ = self.model_.fit(disp=False)
        return self

    def predict(self, X):
        # X can be a 2D array (n_samples, 1) or 1D array of timestamps
        # Convert to 1D array of pandas Timestamps
        dates = pd.to_datetime(np.array(X).flatten())
        preds = []
        for date in dates:
            pred = self.model_fit_.predict(start=date, end=date)
            preds.append(pred.iloc[0])
        return np.array(preds)

# Espacio de búsqueda
param_space = {
    'order': Categorical([(1,1,1), (2,1,1), (1,1,2)]),
    'seasonal_order': Categorical([(1,1,0,4), (1,1,1,4), (2,1,0,4)])
}

# Prepara los datos para sklearn (X: fechas, y: valores)
X_train_sklearn = train_data.index.values.reshape(-1, 1)
y_train_sklearn = train_data.values

opt = BayesSearchCV(
    SARIMAXWrapper(),
    param_space,
    n_iter=10,
    cv=[(np.arange(len(X_train_sklearn)), np.arange(len(X_train_sklearn)))],  # No split, solo fit
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=1
)

opt.fit(X_train_sklearn, y_train_sklearn)

# Mejor modelo
best_model = opt.best_estimator_

# Predicción para diciembre 2019
pred_201912 = best_model.predict(np.array([start_index]))[0]

print(f"Predicción optimizada para product_id={i} en 2019-12: {pred_201912:.5f}")
if not test_data.empty:
    print(f"Valor real: {test_data.iloc[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
sarimax_preds = []
sarimax_real = []
product_ids = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    if len(serie) < 8 or 201912 not in serie.index:
        continue  # Necesitamos suficientes datos y valor real en 201912

    # Convertir el índice a fechas para SARIMAX
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')

    # Entrenar hasta octubre 2019
    train_data = serie[serie.index <= '2019-10-31']
    test_data = serie[serie.index == '2019-12-01']

    try:
        model = SARIMAX(train_data, order=(1,1,1), seasonal_order=(1,1,0,4),
                        enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        pred = model_fit.predict(start=pd.to_datetime('2019-12-01'), end=pd.to_datetime('2019-12-01'))
        sarimax_preds.append(pred.iloc[0])
        sarimax_real.append(test_data.iloc[0])
        product_ids.append(prod_id)
    except Exception:
        continue

df_sarimax_pred = pd.DataFrame({
    'product_id': product_ids,
    'sarimax_pred_201912': sarimax_preds,
    'real_201912': sarimax_real
})

df_sarimax_pred.head()


In [ ]:
# Calcula el error porcentual absoluto medio (MAPE) entre las predicciones SARIMAX y los valores reales
mape_sarimax = np.mean(np.abs((df_sarimax_pred['real_201912'] - df_sarimax_pred['sarimax_pred_201912']) / df_sarimax_pred['real_201912'])) * 100
print(f"Error porcentual absoluto medio (MAPE) SARIMAX: {mape_sarimax:.2f}%")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


# Selecciona el producto i (por ejemplo, product_id = 20001)
i = 20003
serie = pivot_df[pivot_df['product_id'] == i].iloc[0, 1:]  # Excluye la columna product_id
serie = serie.dropna()

# Prepara los datos: X = mes (como entero), y = tn
X = np.array(serie.index.astype(int)).reshape(-1, 1)
y = serie.values

# Train: hasta octubre 2019 (201910)
train_mask = X.flatten() <= 201910
X_train, y_train = X[train_mask], y[train_mask]

# Test: diciembre 2019 (201912)
test_mask = X.flatten() == 201912
X_test, y_test = X[test_mask], y[test_mask]

# Modelo simple: regresión lineal sobre el tiempo
# model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
# model = RandomForestRegressor(random_state=42)
model = RandomForestRegressor(random_state=42, n_estimators=250, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True)
model = RandomForestRegressor(random_state=42, n_estimators=250, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True)

# model = LinearRegression()

model.fit(X_train, y_train)

# Predicción para diciembre 2019 (201912)
pred_201912 = model.predict(np.array([[201912]]))[0]

print(f"Predicción para product_id={i} en 2019-12: {pred_201912:.5f}")
if len(y_test) > 0:
    print(f"Valor real: {y_test[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
from skopt.space import Integer, Categorical
from skopt import BayesSearchCV
iter = 0

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'n_estimators': Categorical([50, 100, 150, 200, 250, 300]),#Integer(50, 400),
    'max_depth': Categorical([1, 5, 10, 20]),
    'min_samples_split':  Categorical([2, 4, 6, 8, 10]),
    'min_samples_leaf':  Categorical([1, 3, 5, 7]),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True])
}


# Optimización bayesiana con logging
opt = BayesSearchCV(
    RandomForestRegressor(random_state=42),
    param_space,
    n_iter=100,
    cv=3,
    n_jobs=1,
    random_state=42,
    scoring='neg_mean_squared_error',
    verbose=1
)

opt.fit(X_train, y_train)
model = opt.best_estimator_

# Predicción para diciembre 2019 (201912)
pred_201912 = model.predict(np.array([[201912]]))[0]

print(f"Predicción para product_id={i} en 2019-12: {pred_201912:.5f}")
if len(y_test) > 0:
    print(f"Valor real: {y_test[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
# Creamos un diccionario para almacenar las predicciones
predicciones = {}

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]  # Excluye la columna product_id
    serie = serie.dropna()
    if len(serie) < 2:
        continue  # No se puede ajustar un modelo con menos de 2 puntos

    X_prod = np.array(serie.index.astype(int)).reshape(-1, 1)
    y_prod = serie.values

    # Train: hasta octubre 2019 (201910)
    train_mask = X_prod.flatten() <= 201910
    X_train_prod, y_train_prod = X_prod[train_mask], y_prod[train_mask]

    # Modelo simple: regresión lineal sobre el tiempo
    if len(X_train_prod) < 2:
        continue  # No se puede ajustar un modelo con menos de 2 puntos

    # model_prod = LinearRegression()
    model_prod = RandomForestRegressor(random_state=42, n_estimators=250, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True)
    model_prod.fit(X_train_prod, y_train_prod)

    # Predicción para diciembre 2019 (201912)
    pred_201912 = model_prod.predict(np.array([[201912]]))[0]
    predicciones[prod_id] = pred_201912

# Convertimos el diccionario a DataFrame para visualizar
df_predicciones = pd.DataFrame(list(predicciones.items()), columns=['product_id', 'pred_201912'])
df_predicciones.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calcula el accuracy de las predicciones usando el error absoluto medio (MAE) y el error cuadrático medio (RMSE)
# Solo para productos que tienen valor real en 201912

# Extraer los valores reales de 201912 para cada producto
valores_reales = df[df['periodo'] == 201912][['product_id', 'tn']]

# Unir con las predicciones
df_eval = pd.merge(df_predicciones, valores_reales, on='product_id', how='inner')

# Calcular métricas

mae = mean_absolute_error(df_eval['tn'], df_eval['pred_201912'])
rmse = mean_squared_error(df_eval['tn'], df_eval['pred_201912'], squared=False)

print(f"MAE (Error absoluto medio): {mae:.4f}")
print(f"RMSE (Raíz del error cuadrático medio): {rmse:.4f}")

In [ ]:
df_eval.head(20)

In [ ]:
mape = np.mean(np.abs((df_eval['tn'] - df_eval['pred_201912']) / df_eval['tn'])) * 100
print(f"MAPE (Error porcentual absoluto medio): {mape:.2f}%")

In [ ]:
# Desvío promedio porcentual de las predicciones respecto a los valores reales
desvio_promedio = np.mean(np.abs(df_eval['pred_201912'] - df_eval['tn']) / df_eval['tn']) * 100
print(f"Desvío promedio porcentual: {desvio_promedio:.2f}%")

In [ ]:


# Selecciona la serie temporal del producto 20001
serie_20001 = pivot_df[pivot_df['product_id'] == 20001].iloc[0, 1:]  # Excluye la columna product_id
serie_20001 = serie_20001.dropna()

# Convierte el índice a periodo de fecha para la serie temporal
serie_20001.index = pd.to_datetime(serie_20001.index.astype(str), format='%Y%m')

# Ajusta el modelo SARIMAX considerando estacionalidad trimestral (4 meses por estación)
model_sarimax = SARIMAX(serie_20001, order=(1,1,1), seasonal_order=(1,1,1,4))
result_sarimax = model_sarimax.fit(disp=False)

# Predicción para diciembre 2019
pred_sarimax = result_sarimax.get_prediction(start=pd.to_datetime('2019-12-01'), end=pd.to_datetime('2019-12-01'))
pred_value = pred_sarimax.predicted_mean.iloc[0]

print(f"Predicción SARIMAX para product_id=20001 en 2019-12: {pred_value:.5f}")

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarimax_preds = []
sarimax_real = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    if len(serie) < 8 or 201912 not in serie.index:
        continue  # Necesitamos suficientes datos y valor real en 201912

    # Convertir el índice a fechas para SARIMAX
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')

    # SARIMAX simple, estacionalidad trimestral (4)
    try:
        model = SARIMAX(serie.iloc[:-1], order=(1,1,1), seasonal_order=(1,1,1,4))
        result = model.fit(disp=False)
        pred = result.get_prediction(start=serie.index[-1], end=serie.index[-1])
        pred_value = pred.predicted_mean.iloc[0]
        real_value = serie.iloc[-1]
        sarimax_preds.append(pred_value)
        sarimax_real.append(real_value)
    except Exception as e:
        continue  # Si falla el ajuste, lo salteamos

sarimax_preds = np.array(sarimax_preds)
sarimax_real = np.array(sarimax_real)
sarimax_mape = np.mean(np.abs((sarimax_real - sarimax_preds) / sarimax_real)) * 100
print(f"MAPE SARIMAX (Error porcentual absoluto medio): {sarimax_mape:.2f}%")

In [ ]:
# Suma total de toneladas reales para diciembre 2019
total_real_dic2019 = valores_reales['tn'].sum()

# Suma total de toneladas predichas para diciembre 2019 (usar df_predicciones)
total_pred_dic2019 = df_predicciones['pred_201912'].sum()

print(f"Suma total real de toneladas en diciembre 2019: {total_real_dic2019:.2f}")
print(f"Suma total predicha de toneladas en diciembre 2019: {total_pred_dic2019:.2f}")

In [ ]:
# Predicción Seasonal Naive Forecast para diciembre 2019 (2019-12-01)
# Usamos el valor de diciembre 2018 (2018-12-01) como predicción para diciembre 2019

# Proyección lineal simple usando solo los valores de 2017-12-01 y 2018-12-01 para cada producto
linear_simple_preds = []
linear_simple_real = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    # Convertir el índice a fechas
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
    # Necesitamos valores para 2017-12-01, 2018-12-01 y 2019-12-01
    if (
        pd.Timestamp('2017-12-01') in serie.index and
        pd.Timestamp('2018-12-01') in serie.index and
        pd.Timestamp('2019-12-01') in serie.index
    ):
        x = np.array([201712, 201812]).reshape(-1, 1)
        y = np.array([serie.loc[pd.Timestamp('2017-12-01')], serie.loc[pd.Timestamp('2018-12-01')]])
        model = LinearRegression()
        model.fit(x, y)
        pred = model.predict(np.array([[201912]]))[0]
        real = serie.loc[pd.Timestamp('2019-12-01')]
        linear_simple_preds.append(pred)
        linear_simple_real.append(real)

linear_simple_preds = np.array(linear_simple_preds)
linear_simple_real = np.array(linear_simple_real)
linear_simple_mape = np.mean(np.abs((linear_simple_real - linear_simple_preds) / linear_simple_real)) * 100
print(f"MAPE Linear Simple (Error porcentual absoluto medio): {linear_simple_mape:.2f}%")

In [ ]:
# La predicción para product_id=20001 en 2019-12 usando el modelo de regresión lineal ya fue calculada en la celda 16:
# pred_201912 = model.predict(np.array([[201912]]))[0]

print(linear_simple_real)

In [ ]:

# pmdarima.auto_arima: Esta biblioteca puede encontrar automáticamente los mejores órdenes (p,d,q)(P,D,Q,S) para cada serie individual,
# basándose en criterios como AIC o BIC. Esto podría mejorar la precisión por producto, pero aumentaría el tiempo de cómputo.

# Python

# Ejemplo con pmdarima para el caso actual

autoarima_preds = []
autoarima_real = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    # Necesitamos al menos 8 datos y valor real en 201912
    if len(serie) < 8 or 201912 not in serie.index:
        continue

    # Convertir el índice a fechas
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')

    # Usar todos los datos menos el último (2019-12-01) para entrenar
    train_serie = serie.iloc[:-1]
    try:
        auto_model = pm.auto_arima(
            train_serie,
            start_p=1, start_q=1,
            max_p=3, max_q=3, m=4,
            start_P=0, seasonal=True,
            d=1, D=1, trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True
        )
        pred = auto_model.predict(n_periods=1)[0]
        real = serie.iloc[-1]
        autoarima_preds.append(pred)
        autoarima_real.append(real)
    except Exception as e:
        continue

autoarima_preds = np.array(autoarima_preds)
autoarima_real = np.array(autoarima_real)
autoarima_mape = np.mean(np.abs((autoarima_real - autoarima_preds) / autoarima_real)) * 100
print(f"MAPE Auto-ARIMA (Error porcentual absoluto medio): {autoarima_mape:.2f}%")

In [ ]:
# Crear un DataFrame con las predicciones y los valores reales del método seasonal naive
df_seasonal_naive = pd.DataFrame({
    'preds': autoarima_preds,
    'real': autoarima_real
})

df_seasonal_naive.head()